In [ ]:
# !pip install html5lib
# !pip install lxml
# !pip install beautifulSoup4

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [ ]:
url = "https://search.shopping.naver.com/catalog/21098473707"
url = "https://search.shopping.naver.com/api/review?isNeedAggregation=N&nvMid=21098473707&page=3&pageSize=20&sortType=QUALITY"
print(url)

In [ ]:
def find_review_page(url, page_no):
    headers = {
        "path": "/api/review?isNeedAggregation=N&nvMid=21098473707&page=1&pageSize=20&sortType=QUALITY",
        "referer":"https://search.shopping.naver.com/catalog/21098473707",
        "user-agent": "Mozilla/5.0 ",
        "sbth": "ba14d435a6a7a2ac56937a81aee1e8a21066bd2959d593905a677a93bcb123eb2f85cb0d7e74ab40d4e163c9015d1cbe"
    }
    response = requests.get(url, headers=headers)
    return response

In [ ]:
res_json = response.json()
res_json["totalCount"]

In [ ]:
pd.DataFrame(res_json["reviews"]).shape

In [ ]:
review_list = []
for page_no in range(1, 5):
    response = find_review_page(url, page_no)
    review_list.append(pd.DataFrame(response.json()["reviews"]))

In [ ]:
df_review = pd.concat(review_list)
df_review.shape

In [ ]:
df_review.head(1)

In [ ]:
reviews = " ".join(df_review["title"].tolist())
reviews[:100]

In [ ]:
import networkx as nx
from collections import Counter
import re

# 텍스트 데이터에서 단어를 추출합니다.
text = "Python is an interpreted, high-level, general-purpose programming language."
text = reviews
text = re.sub("[0-9]", "", text)
words = text.lower().split()

# 각 단어의 빈도수를 계산합니다.
word_counts = Counter(words)

# 단어 간의 연결을 만듭니다.
edges = [(w1, w2, {'weight': count}) for w1, count1 in word_counts.items() 
         for w2, count2 in word_counts.items() 
         for count in [len(set(w1) & set(w2))] if count > 0 and w1 != w2]

# 네트워크 그래프를 생성합니다.
G = nx.Graph()

# 그래프에 노드와 엣지를 추가합니다.
for word, count in word_counts.items():
    G.add_node(word, count=count)
for edge in edges:
    G.add_edge(*edge[:2], **edge[2])

# 그래프에서 중심성 계산을 수행합니다.
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

# 중심성 값을 출력합니다.
for word in word_counts:
    print(f"{word}: degree centrality={degree_centrality[word]:.3f}, closeness centrality={closeness_centrality[word]:.3f}, betweenness centrality={betweenness_centrality[word]:.3f}")


스트 데이터를 가지고 단어 빈도수를 계산한 후, 단어 간의 연결을 만들어 NetworkX 그래프 객체를 생성합니다. 이 그래프에서 중심성 지수(degree, closeness, betweenness centrality)를 계산하고 출력합니다.

위 코드에서는 단어 간의 연결을 만들기 위해 두 단어의 길이가 1 이상인 경우에만 연결을 생성합니다. 이 연결은 두 단어에 공통으로 존재하는 문자의 수로 가중치를 부여합니다. 이렇게 생성된 그래프에서 중심성 지수를 계산하면, 각 단어의 네트워크에서의 중요도를 파악할 수 있습니다.
입력된 텍스트 데이터에서 추출한 단어 간의 연결과 중심성 지수가 출력됩니다.

In [ ]:
# 그래프를 시각화합니다.
plt.figure(figsize=(20,15))
pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_nodes(G, pos, node_size=[count * 200 for count in word_counts.values()], node_color='lightblue')
nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_labels(G, pos, font_size=12, font_family="AppleGothic")
plt.axis('off')
plt.show()
